In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import openslide
import skimage.io
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm



In [ ]:
local_weights_file = '..input/Wajans/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
print(local_weights_file)

In [ ]:
dtypes={'isup_grade' : 'uint8',
       'gleason_score' : 'uint8'}
BASE_PATH = '../input/prostate-cancer-grade-assessment'
data_dir = f'{BASE_PATH}/train_images'
train_df = pd.read_csv(f'{BASE_PATH}/train.csv')
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')
#train.index[0:15]
test_df.head()

In [ ]:
train_df=train_df[:5000]
train_df.shape# WE HAVE GOT 10616 EXAMPLES IN TOTAL

In [ ]:
def preprocess_image(image_path, desired_size=224):
    biopsy = openslide.OpenSlide(image_path)
    im = np.array(biopsy.get_thumbnail(size=(desired_size,desired_size)))
    im = Image.fromarray(im)
    im = im.resize((desired_size,desired_size)) 
    im = np.array(im)
    return im

In [ ]:
N = train_df.shape[0]
x_train = np.empty((N, 224, 224, 3), dtype=np.float)
for i, image_id in enumerate(tqdm(train_df['image_id'])):
    x_train[i, :, :, :] = preprocess_image(f'../input/prostate-cancer-grade-assessment/train_images/{image_id}.tiff')

In [ ]:
print(type(x_train))

In [ ]:
N = test_df.shape[0]
x_test = np.empty((N, 150, 150, 3), dtype=np.float)
for i, image_id in enumerate(tqdm(test_df['image_id'])):
    x_test[i, :, :, :] = preprocess_image(f'../input/prostate-cancer-grade-assessment/train_images/{image_id}.tiff')

print(x_test.shape)


In [ ]:

print(x_train.shape)

In [ ]:

y_train = pd.get_dummies(train_df['isup_grade']).values
y_train =y_train[0:5000]
y_train_multi = np.empty(y_train.shape, dtype=y_train.dtype)
y_train_multi[:, 5] = y_train[:, 5]

for i in range(4, -1, -1):
    y_train_multi[:, i] = np.logical_or(y_train[:, i], y_train_multi[:, i+1])

# ONE HOT ENCODING
'''
from keras.utils.np_utils import to_categorical
y_train = pd.get_dummies(train_df['isup_grade']).values
y_train = y_train[0:5000]
y_train.shape
y_train_multi = to_categorical(y_train,6)'''

In [ ]:
del train_df
del test_df
y_train_multi.shape

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train_multi, 
    test_size=0.20, 
    random_state=1250
)

In [ ]:
print("x_train shape:",x_train.shape)
print("y_train shape:",y_train.shape)
print("x_val shape:", x_val.shape)
print("y_val shape:",y_val.shape)
#print("x_val[6]",x_val[6])
#print('y_val[6]',y_val[6])
#print(x_val[4])
#print(y_val[4])

In [ ]:
# STANDARD CONVOLUTIONAL NEURAL NETWORK
'''
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(6, activation='sigmoid')
])'''

In [ ]:
'''from tensorflow.keras.optimizers import RMSprop,Adam
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=0.00010409613402110064),
              metrics=['accuracy'])'''

In [ ]:
print(tf.__version__)

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.applications import imagenet_utils
from keras.layers import BatchNormalization

from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape=(224,224,3),
                         include_top=False,
                         pooling=max,
                         weights=None)

#pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable=False

last_layer = pre_trained_model.get_layer('mixed7')
print("Output layer shape:",last_layer.output_shape )
last_output = last_layer.output
print("chala")

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow as tf
x = layers.Flatten()(last_output)

x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)

x=layers.Dense(6, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch/2))
optimizer = tf.keras.optimizers.SGD(lr=1e-8)
model.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
print(model)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def train_datagen(): return ImageDataGenerator(
    rescale =1./225,
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')


data_generator = train_datagen().flow(x_train, y_train,
                                      batch_size=128, seed=2019)



In [ ]:
print(x_val[0])

In [ ]:
history = model.fit_generator(
    data_generator,
    steps_per_epoch=4,
    epochs=20,
    verbose=1,
    callbacks=[lr_schedule],
    validation_data=(x_val, y_val))

In [ ]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['accuracy', 'val_accuracy']].plot()

In [ ]:
if os.path.exists('../input/prostate-cancer-grade-assessment/test_images'):
    print("test images found.")
    y_test = model.predict(x_test)
    y_test = y_test > 0.37757874193797547
    y_test = y_test.astype(int).sum(axis=1) - 1
    test_df['isup_grade'] = y_test
    test_df = test_df[["image_id","isup_grade"]]
    test_df.to_csv('submission.csv',index=False)